In [1]:

from keras import backend as K
import matplotlib.pyplot as plt
from numpy import array
import numpy as np
import tensorflow.keras.layers as tfl
import tensorflow as tf
import tensorflow_datasets as tfds
import json
import os, time
from numpy import argmax

Using TensorFlow backend.


In [2]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [3]:
tf.__version__

'2.1.0'

In [7]:

tokenizer = tfds.features.text.Tokenizer(alphanum_only=False)
vocabulary_set = set()
all_genres=set()
max_entries=float("inf")
entries=0

In [8]:
genres_list=[]
scripts=[]
for r, d, f in os.walk("./data"):
    for file in f:
        if '.json' in file:  # only load 10 jsons for now

            with open(os.path.join(r, file), mode='r', encoding='utf-8-sig') as prep_file:
                try:
                    script_data = json.load(prep_file)
                    genres=script_data["genres"] if script_data["genres"] else []
                    if None in genres:
                        genres.remove(None)
                    all_genres.update(genres)
                    n=0
                    script_tokenized=tokenizer.tokenize(script_data["script"])
                    vocabulary_set.update(script_tokenized)
                    genres_list.append(genres)
                    scripts.append(script_tokenized)
                    entries+=1
                    if entries>max_entries:
                        break
                    n+=1

                    if entries>max_entries:
                        break
                except:
                    print("error on file: ",prep_file)


error on file:  <_io.TextIOWrapper name='./data/test.json' mode='r' encoding='utf-8-sig'>


In [28]:
scripts_encoded=[]
site_text_encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)
for script in scripts:
    script_joined=" ".join(script)

    scripts_encoded.append(site_text_encoder.encode(script_joined))

scripts_padded = tf.keras.preprocessing.sequence.pad_sequences(scripts_encoded,
                                                                padding='post',maxlen=site_words)
all_genres_list=list(all_genres)
genres_map=[]
for genres in genres_list:
    active_cats=[1 if genre in genres else 0 for genre in all_genres_list ]
    genres_map.append(np.array(active_cats))
scripts_stacked = tf.stack(scripts_padded)
genres_stacked = tf.stack(genres_map)
sites_dataset = tf.data.Dataset.from_tensor_slices(
    (scripts_stacked,genres_stacked))

In [29]:
genres_stacked.shape[1]

26

In [36]:

BUFFER_SIZE = 60000
BATCH_SIZE = 32
EPOCHS = 2
noise_dim = 100
# Batch and shuffle the data
train_dataset = sites_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# Length of the vocabulary in chars
vocab_size = len(vocabulary_set)
genres_size=genres_stacked.shape[1]
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 128
def make_classifier():
    model = tf.keras.Sequential([
        tfl.Embedding(vocab_size, embedding_dim, input_shape=(None,)),
        tfl.GRU(rnn_units),
        tfl.Dense(genres_size*16),
        tfl.Dense(genres_size,activation="elu")
    ])

    return model


In [37]:
model=make_classifier()
model.summary()
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=["mse","mae"])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 256)         72981504  
_________________________________________________________________
gru_6 (GRU)                  (None, 128)               148224    
_________________________________________________________________
dense_13 (Dense)             (None, 416)               53664     
_________________________________________________________________
dense_14 (Dense)             (None, 26)                10842     
Total params: 73,194,234
Trainable params: 73,194,234
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(train_dataset, epochs=9)

Train for 36 steps
Epoch 1/3
36/36 [==============================] - 33s 912ms/step - loss: 0.3874 - mse: 0.1237 - mae: 0.2631
Epoch 2/3
36/36 [==============================] - 32s 886ms/step - loss: 0.2414 - mse: 0.0740 - mae: 0.1465
Epoch 3/3
36/36 [==============================] - 32s 885ms/step - loss: 0.2341 - mse: 0.0726 - mae: 0.1447


In [53]:
#test index
index=19
test_script=scripts[index]
test_encoded=site_text_encoder.encode("".join(test_script))
test_padded = tf.keras.preprocessing.sequence.pad_sequences([test_encoded],
                                                                padding='post',maxlen=site_words)
predictions=model(test_padded).numpy()
predictions=predictions/predictions.max()
print(predictions,predictions.shape)
for i,genre in enumerate(all_genres_list):
    if predictions[0,i] > 0.5:
        print("Movie in", genre)
print("wanted:",genres_list[index])

[[0.7425241  0.7471364  0.30532533 0.01257539 0.01210212 0.56437695
  0.08027765 0.00756055 0.00885526 0.00907336 0.06766494 0.05849902
  0.07031178 0.37965572 0.20553645 0.01796708 0.00775568 0.4442241
  0.01784999 1.         0.4018589  0.40778035 0.43002096 0.06461545
  0.01046722 0.01085936]] (1, 26)
Movie in Action
Movie in Thriller
Movie in Comedy
Movie in Drama
wanted: ['Action', 'Adventure', 'Sci-Fi']
